<a href="https://colab.research.google.com/github/alti-tude/video-slide-matching/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install scikit-image

  Using cached https://files.pythonhosted.org/packages/3b/ee/edbfa69ba7b7d9726e634bfbeefd04b5a1764e9e74867ec916113eeaf4a1/skimage-0.0.tar.gz
Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-install-ny2fe2z2/skimage/


In [1]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
import numpy as np
import cv2
from matplotlib import pylab as plt
from math import floor, ceil

root = '/content/gdrive/My Drive/data/Data/new_test/'
frame_root = root + 'frames/'
ppt_root = root + 'ppts/'


In [0]:
from IPython.display import HTML, display

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
        <p> {value}/{max}</p>
    """.format(value=value, max=max))


In [0]:
from skimage.measure import compare_ssim as ssim
cache = {}

def compare(frame, ppt):
    global cache
    print(frame, ppt)
    if frame in cache:
        frame = cache[frame].copy()
    else:
        cache[frame] = cv2.imread(frame, 0)
        frame = cache[frame].copy()
        
    if ppt in cache:
        ppt = cache[ppt].copy()
    else:
        cache[ppt] = cv2.imread(ppt,0)
        ppt = cache[ppt].copy()
        
#     plt.subplot(121), plt.imshow(frame, cmap='gray')
#     plt.subplot(122), plt.imshow(ppt, cmap='gray')
#     plt.show()

    n = min(ppt.shape[0], frame.shape[0])
    m = min(ppt.shape[1], frame.shape[1])
    
    frame = frame[int((frame.shape[0]-n)/2):frame.shape[0] - ceil((frame.shape[0]-n)/2), int((frame.shape[1]-m)/2):frame.shape[1] - ceil((frame.shape[1]-m)/2)]
    ppt = ppt[int((ppt.shape[0]-n)/2):ppt.shape[0] - ceil((ppt.shape[0]-n)/2), int((ppt.shape[1]-m)/2):ppt.shape[1] - ceil((ppt.shape[1]-m)/2)]
    
    frame = frame.astype(float)
    ppt = ppt.astype(float)
    
    
    mean_f = np.mean(frame)
    mean_p = np.mean(ppt)
    sd_f = np.std(frame)
    sd_p = np.std(ppt)
    
    ###### for implementing ssim #############
#     sd_pf = np.sqrt(np.sum(frame*ppt)/(n*m-1))
#     print((4*mean_f*mean_p*sd_pf)/((mean_f**2+mean_p**2)*(sd_f**2+sd_p**2)))
#     return (4*mean_f*mean_p*sd_pf)/((mean_f**2+mean_p**2)*(sd_f**2+sd_p**2))

    ############# normxcorr ###################
    return np.sum((frame-mean_f)*(ppt-mean_p))/(sd_f*sd_p)

In [0]:
ppts = os.listdir(ppt_root)
frames = os.listdir(frame_root)
op = open(root+'op3.txt', 'w')

out = display(progress(0, len(frames)), display_id=True)
cache = {}
co = 0

for frame in frames:
    ma = 0
    best_match = -1
    for ppt in ppts:
        corr = compare(frame_root+frame, ppt_root+ppt)
        
        if corr>ma:
            ma = corr
            best_match = ppt
            
    op.write("%s %s\n"%(frame, best_match))
    out.update(progress(co+1, len(frames)))
    co+=1
    
op.close()

In [0]:
!diff /content/gdrive/My\ Drive/data/Data/new_test/op2.txt /content/gdrive/My\ Drive/data/Data/new_test/op3.txt

In [0]:
compare(root+'../tmp/frame.jpg', root+'../tmp/correct_match.jpg')